In [266]:
import pandas as pd
import numpy as np
import sklearn as sk

In [267]:
pricing_requests = pd.read_csv("AVN_Data.csv",index_col =0)

In [268]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder

In [269]:
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])

In [270]:
y = pricing_requests["price"]
X = pricing_requests.loc[ : , pricing_requests.columns != 'price']

In [271]:
X

,hotel_id,stock,city,date,language,mobile,avatar_id,group,brand,parking,pool,children_policy
0,969,56,amsterdam,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0
23,909,56,madrid,44,dutch,0,111612,Yin Yang,8 Premium,1,0,0
113,789,56,madrid,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0
115,915,56,madrid,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0
20,987,56,rome,44,dutch,0,111612,Accar Hotels,Ibas,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
131,29,4,paris,5,finnish,1,303616,Accar Hotels,Marcure,1,0,0
39,833,74,vienna,35,irish,1,303616,Morriott International,CourtYord,1,0,1
39,833,74,vienna,35,irish,1,303616,Morriott International,CourtYord,1,0,1
39,833,74,vienna,35,irish,1,303616,Morriott International,CourtYord,1,0,1


In [281]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import QuantileTransformer, StandardScaler


numeric_features = ["hotel_id", "stock","date"]
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_features = ["city","language","mobile","avatar_id","group","brand","parking","pool","children_policy"]
categorical_transformer = TargetEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 500, num = 3)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 3)]
max_depth.append(None)
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Create the random grid
random_grid = {'randomforestregressor__n_estimators': n_estimators,
               'randomforestregressor__max_depth': max_depth,
               'randomforestregressor__min_samples_leaf': min_samples_leaf}

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('randomforestregressor', RandomForestRegressor())
])

# Create the randomized search object
random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Fit the randomized search object to the training data
random_search.fit(X, y)

# Get the best combination of hyperparameters
best_params = random_search.best_params_

/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 36 is smaller than n_iter=100. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/pipeline.py", line 699, in score
    return self.steps[-1][1].score(Xt, y, **score_params)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/base.py", line 720, in score
    y_pred = self.predict(X)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 991, in predict
    X = self._

/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/pipeline.py", line 699, in score
    return self.steps[-1][1].score(Xt, y, **score_params)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/base.py", line 720, in score
    y_pred = self.predict(X)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 991, in predict
    X = self._

/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/pipeline.py", line 699, in score
    return self.steps[-1][1].score(Xt, y, **score_params)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/base.py", line 720, in score
    y_pred = self.predict(X)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 991, in predict
    X = self._

...

/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score

/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/ca

/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/ca

/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/ca

/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 429, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/pipeline.py", line 699, in score
    return self.steps[-1][1].score(Xt, y, **score_params)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/base.py", line 720, in score
    y_pred = self.predict(X)
  File "/usr/local/insa/anaconda/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 991, in predict
    X = self._

In [273]:
best_params

{'randomforestregressor__n_estimators': 200,
 'randomforestregressor__min_samples_leaf': 1,
 'randomforestregressor__max_depth': 10}

In [274]:
?RandomForestRegressor

In [275]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import QuantileTransformer, StandardScaler


numeric_features = ["hotel_id", "stock","date"]
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_features = ["city","language","mobile","group","brand","parking","pool","children_policy"]
categorical_transformer = TargetEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)



# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('randomforestregressor', RandomForestRegressor(n_estimators = 300))
])

pipeline.fit(X, y)

/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/brosset/.local/lib/python3.9/site-packages/ca

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['hotel_id', 'stock',
                                                   'date']),
                                                 ('cat',
                                                  TargetEncoder(handle_unknown='ignore'),
                                                  ['city', 'language', 'mobile',
                                                   'group', 'brand', 'parking',
                                                   'pool',
                                                   'children_policy'])])),
                ('randomforestregressor',
                 RandomForestRegressor(n_estimators=300))])

In [282]:
test = pd.read_csv("test_set.csv",index_col =0)

test = test.join(hotels, on=['hotel_id', 'city'])
test = test.drop(["order_requests"],axis =1)

In [283]:
test

,city,date,language,mobile,avatar_id,hotel_id,stock,group,brand,parking,pool,children_policy
index,,,,,,,,,,,,
0,vilnius,21,romanian,0,1,161,46,Boss Western,J.Halliday Inn,1,0,0
1,vilnius,21,romanian,0,1,187,32,Accar Hotels,Marcure,1,1,0
2,vilnius,21,romanian,0,1,279,12,Independant,Independant,1,0,0
3,vilnius,21,romanian,0,1,395,10,Accar Hotels,Ibas,0,0,0
4,vilnius,21,romanian,0,1,488,42,Accar Hotels,Safitel,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6639,rome,5,irish,0,794,987,1,Accar Hotels,Ibas,1,0,0
6640,vienna,1,irish,1,794,26,1,Accar Hotels,Marcure,1,0,0
6641,vienna,1,irish,1,794,263,0,Boss Western,Boss Western,1,0,0


In [284]:
A = random_search.predict(test)

In [285]:
Submission = pd.DataFrame(A)

Submission.to_csv("submission2023.csv",header = ["price"],index_label="index")

In [286]:
Submission

,0
0,144.354424
1,253.727327
2,62.745394
3,76.669059
4,251.470723
...,...
6639,79.864281
6640,245.269449
6641,110.918834
6642,180.623797
